<a href="https://colab.research.google.com/github/Vavilexxx/sql-29-final/blob/main/%D0%94%D0%97_stat_basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашнее задание к лекции "Базовые понятия статистики"

In [ ]:
import pandas as pd
from numpy import NaN

Будем осуществлять работу с непростым [набором данных](https://raw.githubusercontent.com/obulygin/pyda_homeworks/master/statistics_basics/horse_data.csv) о состоянии здоровья лошадей, испытывающих кишечные колики. Цель – максимально корректно заполнить пропуски.

### Задание 1. Загрузка данных

Изучить представленный набор данных на основе [описания его столбцов](https://raw.githubusercontent.com/obulygin/pyda_homeworks/master/statistics_basics/horse_data.names), загрузить его и оставить 8 столбцов для дальнейшего изучения: `surgery?`, `Age`, `rectal temperature`, `pulse`, `respiratory rate`, `temperature of extremities`, `pain`, `outcome`.

### Задание 2. Первичное изучение данных

Проанализировать значения по столбцам, рассчитать базовые статистики, найти выбросы.

### Задание 3. Работа с пропусками

Рассчитать количество пропусков для всех выбранных столбцов. Принять и обосновать решение о методе заполнения пропусков по каждому столбцу на основе рассчитанных статистик и возможной взаимосвязи значений в них. Сформировать датафрейм, в котором пропуски будут отсутствовать.

In [ ]:
df = pd.read_csv(r'https://raw.githubusercontent.com/obulygin/pyda_homeworks/master/statistics_basics/horse_data.csv',
                 names=['surgery', 'age', 'rectal_temperature', 'pulse', 'respiratory_rate', 'temperature_of_extremities', 'pain', 'outcome'],
                 usecols=[0, 1, 3, 4, 5, 6, 10, 22])
df.head()

,surgery,age,rectal_temperature,pulse,respiratory_rate,temperature_of_extremities,pain,outcome
0,2,1,38.50,66,28,3,5,2
1,1,1,39.2,88,20,?,3,3
2,2,1,38.30,40,24,1,3,1
3,1,9,39.10,164,84,4,2,2
4,2,1,37.30,104,35,?,?,2


In [ ]:
# Процент пропусков до
(df.isin(['?']).mean() * 100).round(2)

surgery                        0.33
age                            0.00
rectal_temperature            20.00
pulse                          8.00
respiratory_rate              19.33
temperature_of_extremities    18.67
pain                          18.33
outcome                        0.33
dtype: float64

In [ ]:
def get_basic_stats(column, passes='?'):
    print(column.describe())
    print('count: ', column.count())
    print('min: ', column.min())
    print('max: ', column.max())
    print('mode: ', column.mode()[0])
    print('unique: ', sorted(set(column)))
    print('passes: ', column.where(column == passes).count())  # Количество пропусков.


def get_outliers(col):
    q1 = col.quantile(0.25)
    q3 = col.quantile(0.75)
    iqr = q3 - q1
    lower_outliers = q1 - (1.5 * iqr)
    upper_outliers = q3 + (1.5 * iqr)
    print('outliers:', [i for i in col if i < lower_outliers or i > upper_outliers])

In [ ]:
# Столбец surgery (категориальные данные) процент пропусков - 0.33
# Количество пропусков.
# Пропуск едиственный
get_basic_stats(df['surgery'])
# Выбросов нет
# Пропуски заменим на моду
df.loc[df['surgery'].isin(['?']), 'surgery'] = df['surgery'].mode()[0]

print('-' * 15)
get_basic_stats(df['surgery'])

count     300
unique      3
top         1
freq      180
Name: surgery, dtype: object
count:  300
min:  1
max:  ?
mode:  1
unique:  ['1', '2', '?']
passes:  1
---------------
count     300
unique      2
top         1
freq      181
Name: surgery, dtype: object
count:  300
min:  1
max:  2
mode:  1
unique:  ['1', '2']
passes:  0


In [ ]:
# Столбец age (категориальные данные). Пропусков нет.
get_basic_stats(df['age'])
get_outliers(df['age'])
# Есть выбросы в виде 9. Вероятно это ошибка распознавания 2. Заменим 9 на 2
df.loc[df['age'] == 9, 'age'] = 2
get_basic_stats(df['age'])

count    300.000000
mean       1.640000
std        2.173972
min        1.000000
25%        1.000000
50%        1.000000
75%        1.000000
max        9.000000
Name: age, dtype: float64
count:  300
min:  1
max:  9
mode:  1
unique:  [1, 9]
passes:  0
outliers: [9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9]
count    300.000000
mean       1.080000
std        0.271746
min        1.000000
25%        1.000000
50%        1.000000
75%        1.000000
max        2.000000
Name: age, dtype: float64
count:  300
min:  1
max:  2
mode:  1
unique:  [1, 2]
passes:  0


In [ ]:
# Столбец rectal_temperature (непрерывные данные)
# Процент пропусков 20%
df.loc[df['rectal_temperature'].isin(['?']), 'rectal_temperature'] = '0.0'
df['rectal_temperature'] = df['rectal_temperature'].astype(float).round(1)
get_basic_stats(df['rectal_temperature'], 0.0)

# Количество пропусков 60 из 300.
# Заменим пропуски на среднее значение
df.loc[df['rectal_temperature'].isin([0.0]), 'rectal_temperature'] = df['rectal_temperature'].mean().round(1)
# Выбросов нет
get_outliers(df['rectal_temperature'])

print('-' * 15)
get_basic_stats(df['rectal_temperature'], 0.0)

count    300.000000
mean      30.534333
std       15.306684
min        0.000000
25%       37.200000
50%       38.000000
75%       38.500000
max       40.800000
Name: rectal_temperature, dtype: float64
count:  300
min:  0.0
max:  40.8
mode:  0.0
unique:  [0.0, 35.4, 36.0, 36.1, 36.4, 36.5, 36.6, 36.8, 36.9, 37.0, 37.1, 37.2, 37.3, 37.4, 37.5, 37.6, 37.7, 37.8, 37.9, 38.0, 38.1, 38.2, 38.3, 38.4, 38.5, 38.6, 38.7, 38.8, 38.9, 39.0, 39.1, 39.2, 39.3, 39.4, 39.5, 39.6, 39.7, 39.9, 40.0, 40.3, 40.8]
passes:  60
outliers: [30.5, 30.5, 30.5, 30.5, 30.5, 30.5, 30.5, 30.5, 30.5, 30.5, 30.5, 30.5, 30.5, 30.5, 30.5, 30.5, 30.5, 30.5, 30.5, 30.5, 30.5, 30.5, 30.5, 30.5, 30.5, 30.5, 30.5, 30.5, 30.5, 30.5, 30.5, 30.5, 30.5, 30.5, 30.5, 30.5, 30.5, 30.5, 30.5, 30.5, 30.5, 30.5, 30.5, 30.5, 30.5, 30.5, 30.5, 30.5, 30.5, 30.5, 30.5, 30.5, 30.5, 40.8, 30.5, 30.5, 30.5, 30.5, 30.5, 30.5, 30.5]
---------------
count    300.000000
mean      36.634333
std        3.141276
min       30.500000
25%       37.20

In [ ]:
# Столбец pulse (непрерывные данные)
# Процент пропусков 8%
get_basic_stats(df['pulse'])

df.loc[df['pulse'].isin(['?']), 'pulse'] = '0'
df['pulse'] = df['pulse'].astype(int)

# Количество пропусков 24 из 300.
# Вероятно пульс линейно зависит от боли (не всегда) и температуры (часто).
# Заменим пропуски на среднее значение по группе (ректальная температура).
df.loc[df['pulse'].isin([0]), 'pulse'] = df.groupby('rectal_temperature')['pulse'].transform('mean').round()
# Минимальное значение 30, максимальное 184. Выбросы - [164, 150, 160, 184, 150]. Но очевидно, что такие показатели пульса возможны.
get_outliers(df['pulse'])

print('-' * 15)
get_basic_stats(df['pulse'], 0)

count     300
unique     53
top        48
freq       28
Name: pulse, dtype: object
count:  300
min:  100
max:  ?
mode:  48
unique:  ['100', '104', '108', '110', '112', '114', '120', '124', '128', '129', '130', '132', '136', '140', '146', '150', '160', '164', '184', '30', '36', '38', '40', '42', '44', '45', '46', '48', '49', '50', '52', '54', '56', '60', '64', '65', '66', '68', '70', '72', '75', '76', '78', '80', '82', '84', '86', '88', '90', '92', '96', '98', '?']
passes:  24
outliers: [164, 150, 160, 184, 150]
---------------
count    300.000000
mean      70.816667
std       27.765918
min       30.000000
25%       48.000000
50%       60.000000
75%       88.000000
max      184.000000
Name: pulse, dtype: float64
count:  300
min:  30
max:  184
mode:  48
unique:  [30, 36, 38, 40, 42, 44, 45, 46, 48, 49, 50, 52, 53, 54, 56, 57, 59, 60, 64, 65, 66, 68, 70, 72, 75, 76, 78, 80, 82, 84, 86, 88, 90, 92, 96, 98, 100, 104, 108, 110, 112, 114, 120, 124, 128, 129, 130, 132, 136, 140, 146, 150, 160,

In [ ]:
# Столбец respiratory_rate (непрерывные данные)
# Процент пропусков 19.33%
get_basic_stats(df['respiratory_rate'])

df.loc[df['respiratory_rate'].isin(['?']), 'respiratory_rate'] = '0'
df['respiratory_rate'] = df['respiratory_rate'].astype(int)

# Количество пропусков 24 из 300.
# Вероятно пульс может линейно зависить от температуры.
# Заменим пропуски на среднее значение по группе (ректальная температура).
df.loc[df['respiratory_rate'].isin([0]), 'respiratory_rate'] = df.groupby('rectal_temperature')['respiratory_rate'].transform('mean').round()
# Минимальное значение 8, максимальное 96. Выбросы - [84, 96, 72, 80, 80, 68, 96, 66, 68, 90, 80, 70, 88, 84, 68, 90, 70]. Но очевидно, что такие показатели ошибочны.
get_outliers(df['respiratory_rate'])

print('-' * 15)
get_basic_stats(df['respiratory_rate'], 0)

count     300
unique     41
top         ?
freq       58
Name: respiratory_rate, dtype: object
count:  300
min:  10
max:  ?
mode:  ?
unique:  ['10', '12', '13', '14', '15', '16', '18', '20', '21', '22', '23', '24', '25', '26', '28', '30', '32', '34', '35', '36', '40', '42', '44', '48', '50', '51', '52', '58', '60', '66', '68', '70', '72', '8', '80', '84', '88', '9', '90', '96', '?']
passes:  58
outliers: [84, 96, 72, 80, 80, 68, 96, 66, 68, 90, 80, 70, 88, 84, 68, 90, 70]
---------------
count    300.000000
mean      28.136667
std       16.856407
min        0.000000
25%       16.000000
50%       24.000000
75%       35.000000
max       96.000000
Name: respiratory_rate, dtype: float64
count:  300
min:  0
max:  96
mode:  16
unique:  [0, 8, 9, 10, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 30, 32, 34, 35, 36, 40, 42, 44, 48, 50, 51, 52, 58, 60, 66, 68, 70, 72, 80, 84, 88, 90, 96]
passes:  3


In [ ]:
# Столбец temperature_of_extremities (категориальные данные)
# Процент пропусков 18.67
get_basic_stats(df['temperature_of_extremities'])

# Выбросов нет.
# Количество пропусков 55.
# Заменим пропуски на моду
df.loc[df['temperature_of_extremities'].isin(['?']), 'temperature_of_extremities'] = df['temperature_of_extremities'].mode()[0]

print('-' * 15)
get_basic_stats(df['temperature_of_extremities'])

count     300
unique      5
top         3
freq      109
Name: temperature_of_extremities, dtype: object
count:  300
min:  1
max:  ?
mode:  3
unique:  ['1', '2', '3', '4', '?']
passes:  56
---------------
count     300
unique      4
top         3
freq      165
Name: temperature_of_extremities, dtype: object
count:  300
min:  1
max:  4
mode:  3
unique:  ['1', '2', '3', '4']
passes:  0


In [ ]:
# Столбуц pain (категориальные данные)
# Процент пропусков 18.33%
get_basic_stats(df['pain'])

# Это субьективный показатель, следовательно вряд ли зависим от других показателей.
# Выбросов нет.
# Количество пропусков 55.
# Заменим пропуски на моду
df.loc[df['pain'].isin(['?']), 'pain'] = df['pain'].mode()[0]

print('-' * 15)
get_basic_stats(df['pain'])

count     300
unique      6
top         3
freq       67
Name: pain, dtype: object
count:  300
min:  1
max:  ?
mode:  3
unique:  ['1', '2', '3', '4', '5', '?']
passes:  55
---------------
count     300
unique      5
top         3
freq      122
Name: pain, dtype: object
count:  300
min:  1
max:  5
mode:  3
unique:  ['1', '2', '3', '4', '5']
passes:  0


In [ ]:
# Столбец outcome (категориальные данные) процент пропусков - 0.33
get_basic_stats(df['outcome'])

# Пропуск едиственный
# Выбросов нет
# Пропуск заменим на моду
df.loc[df['outcome'].isin(['?']), 'outcome'] = df['outcome'].mode()[0]

print('-' * 15)
get_basic_stats(df['outcome'])

count     300
unique      4
top         1
freq      178
Name: outcome, dtype: object
count:  300
min:  1
max:  ?
mode:  1
unique:  ['1', '2', '3', '?']
passes:  1
---------------
count     300
unique      3
top         1
freq      179
Name: outcome, dtype: object
count:  300
min:  1
max:  3
mode:  1
unique:  ['1', '2', '3']
passes:  0


In [ ]:
# Процент пропусков после
(df.isin(['?']).mean() * 100).round(2)

surgery                       0.0
age                           0.0
rectal_temperature            0.0
pulse                         0.0
respiratory_rate              0.0
temperature_of_extremities    0.0
pain                          0.0
outcome                       0.0
dtype: float64

In [ ]:
df.head()

,surgery,age,rectal_temperature,pulse,respiratory_rate,temperature_of_extremities,pain,outcome
0,2,1,38.5,66,28,3,5,2
1,1,1,39.2,88,20,3,3,3
2,2,1,38.3,40,24,1,3,1
3,1,2,39.1,164,84,4,2,2
4,2,1,37.3,104,35,3,3,2


#### ПРИМЕЧАНИЕ
Домашнее задание сдается ссылкой [Google Colab](https://colab.research.google.com/).
Не сможем проверить или помочь, если вы пришлете:
- файлы;
- архивы;
- скриншоты кода.

Все обсуждения и консультации по выполнению домашнего задания ведутся только на соответствующем канале в slack.

##### Как правильно задавать вопросы аспирантам, преподавателям и коллегам
Прежде чем задать вопрос, попробуйте найти ответ в интернете. Навык самостоятельного поиска информации — один из важнейших. Каждый практикующий специалист любого уровня делает это ежедневно.

Сформулируйте вопрос по алгоритму:  
1) Что я делаю?  
2) Какого результата я ожидаю?  
3) Как фактический результат отличается от ожидаемого?  
4) Что я уже попробовал сделать, чтобы исправить проблему?  

По возможности прикрепите к вопросу скриншоты либо ссылки на код. Не выкладывайте все решение, оставляйте только проблемный и воспроизводимый участок кода.